In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from binance.client import Client
import pandas as pd
import time

# Initialize Binance client (no API key needed for public data)
client = Client()

# Define your tickers - Binance uses different symbols, typically without '-USD'
tickers = ["BTCUSDT", "XRPUSDT", "ETHUSDT", "SOLUSDT", "DOGEUSDT", "ADAUSDT", 
           "SHIBUSDT", "DOTUSDT", "BTTUSDT", "LINKUSDT", "ALGOUSDT", "AVAXUSDT",
           "XLMUSDT", "NEARUSDT", "LTCUSDT", "CHZUSDT", "POLUSDT", "GRTUSDT",
           "LRCUSDT", "ARBUSDT", "UNIUSDT", "GALAUSDT", "INJUSDT", "TRXUSDT", "CRVUSDT",
           "ANKRUSDT", "NMRUSDT", "WOOUSDT", "MANAUSDT", "AAVEUSDT", "QNTUSDT", "BCHUSDT",
           "SUSHIUSDT", "APEUSDT", "ZRXUSDT", "ETCUSDT", "KSMUSDT", "SANDUSDT", "IMXUSDT", 
           "1INCHUSDT", "OPUSDT", "ATOMUSDT", "POWRUSDT", "AXSUSDT", "YFIUSDT", 
           "SNXUSDT", "MKRUSDT", "STORJUSDT", "GNOUSDT", "BATUSDT", "REQUSDT", "COMPUSDT", 
           "XTZUSDT", "BNTUSDT", "ENJUSDT", "EOSUSDT"]

# Date range
start_str = "2018-01-01"
end_str = "2024-01-01"

# Helper to get historical data from Binance
def get_binance_ohlcv(symbol, interval, start_str, end_str):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, start_str, end_str))
    if df.empty:
        return None
    df = df.iloc[:, 0:6]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

# Create a dictionary of DataFrames for each ticker
data = {}
for ticker in tickers:
    try:
        print(f"Downloading data for {ticker}...")
        df = get_binance_ohlcv(ticker, Client.KLINE_INTERVAL_1DAY, start_str, end_str)
        if df is not None:
            data[ticker] = df
        time.sleep(0.3)  # To avoid hitting Binance rate limits
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

In [ ]:
import yfinance as yf
btc_df = data['BTCUSDT'][['close']].rename(columns={'close': 'BTC'})
btc_df.index = pd.to_datetime(btc_df.index)
spy = yf.download("SPY", start=start_str, end=end_str)
spy = spy[['Close']].rename(columns={'Close': 'SPY'})
spy.index = pd.to_datetime(spy.index)
spy = spy["SPY"]

In [ ]:
# Create a dictionary of DataFrames for each ticker
start_str = "2024-01-01"
end_str = "2025-06-01"

data_test = {}
for ticker in tickers:
    try:
        print(f"Downloading data for {ticker}...")
        df = get_binance_ohlcv(ticker, Client.KLINE_INTERVAL_1DAY, start_str, end_str)
        if df is not None:
            data_test[ticker] = df
        time.sleep(0.3)  # To avoid hitting Binance rate limits
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

btc_df_test = data_test['BTCUSDT'][['close']].rename(columns={'close': 'BTC'})
btc_df_test.index = pd.to_datetime(btc_df_test.index)
spy_test = yf.download("SPY", start=start_str, end=end_str)
spy_test = spy_test[['Close']].rename(columns={'Close': 'SPY'})
spy_test.index = pd.to_datetime(spy_test.index)
spy_test = spy_test["SPY"]

In [ ]:
def slope_func(y):
    # Use np.polyfit to fit a line to the window
    return np.polyfit(np.arange(len(y)), y, 1)[0]

def strategy1_exp_momentum_trend_volatility(df, exp_t=20, ema_t=100, slope_t=14, bb_t=20, bb_width=1.5, mom_thr=0.02, sell_ema=25):
    df = df.copy()

    # Calculate EWMA Momentum
    df['returns'] = df['close'].pct_change()
    df['exp_momentum'] = df['returns'].ewm(span=exp_t, adjust=False).mean()

    # EMA100 Trend Filter
    df['EMA100'] = df['close'].ewm(span=ema_t, adjust=False).mean()
    df['sell_ema'] = df['close'].ewm(span=sell_ema, adjust=False).mean()

    df["slope"] = df["close"].rolling(window=slope_t, min_periods=slope_t).apply(slope_func, raw=True)

    # Volatility Filter using Bollinger Band Width
    rolling_mean = df['close'].rolling(window=bb_t).mean()
    rolling_std = df['close'].rolling(window=bb_t).std()
    df['BB_upper'] = rolling_mean + bb_width * rolling_std
    df['BB_lower'] = rolling_mean - bb_width * rolling_std
    df['BB_width'] = df['BB_upper'] - df['BB_lower']
    df['BB_avg'] = df['BB_width'].rolling(window=bb_t).mean()
    df['volatility_ok'] = df['BB_width'] > df['BB_avg']

    # Generate Signals
    df['signal'] = 0
    # random_sells = np.random.rand(len(df)) < 0.01
    # df.loc[random_sells, 'signal'] = -1
    df.loc[(df['exp_momentum'] > mom_thr) & (df['close'] > df['EMA100']) & df['volatility_ok'] & (df["slope"] > 0.0), 'signal'] = 1
    df.loc[(df['close'] < df['sell_ema']), 'signal'] = -1

    #df["signal"] = np.where(df["pre_signal"].rolling(3).mean() == 1, 1, np.where(df["pre_signal"] == -1, -1, 0))
    # Positions - avoid chained assignment by building list
    # position = 0
    # positions = []
    # for sig in df['signal']:
    #     if sig == 1 and position == 0:
    #         position = 1
    #     elif sig == -1 and position == 1:
    #         position = 0
    #     positions.append(position)
    # df['positions'] = positions

    # Strength: Use exp_momentum value on buy signals; else 0
    df['strength'] = 0.0
    df.loc[df['signal'] == 1, 'strength'] = df.loc[df['signal'] == 1, 'exp_momentum']

    return df


In [ ]:
def backtest_signals(df):
    """
    df: DataFrame with 'close', 'high', 'low', and 'positions' columns.
    'positions' = 1 means holding, 0 means no position.
    Returns a summary dict with metrics and list of trades.
    """
    df = df.copy()

    # set last two rows of df to -1 to handle end of data trades
    df.loc[df.index[-1], "signal"] = -1
    df.loc[df.index[-2], 'signal'] = -1

    trades = []
    entries = []
    exits = []
    entry_idx = None

    for i in range(len(df) - 1):  # avoid index out of bounds for i+1
        # Entry signal detected
        if df['signal'].iloc[i] == 1:
            entry_idx = i  # trade happens next day
            price_idx = i + 1
            entries.append({
                "entry_idx": entry_idx,
                "price_idx": price_idx 
            })

    for i in range(len(df) - 1):  # avoid index out of bounds for i+1
        if df['signal'].iloc[i] == -1:
            exit_idx = i
            price_idx = i + 1
            exits.append({
                "exit_idx": exit_idx,
                "price_idx": price_idx
            })
            
    for e in entries:
        next_exit = next((x for x in exits if x['exit_idx'] > e['entry_idx']), None)
        if next_exit is not None:
            entry_price = df['high'].iloc[e["price_idx"]]
            exit_price = df['low'].iloc[next_exit["price_idx"]]
            entry_date = df.index[e["entry_idx"]]
            exit_date = df.index[next_exit["exit_idx"]]
            ret = (exit_price - entry_price) / entry_price
            hold_time = (exit_date - entry_date).days
            trades.append({
                'entry_date': entry_date,
                'exit_date': exit_date,
                'return': ret,
                'hold_time_days': hold_time,
                'entry_idx': e['entry_idx'],
                'exit_idx': next_exit['exit_idx']
            })

    # # If currently in a position at the end, close it on the next available day
    # if entry_idx is not None and entry_idx < len(df):
    #     entry_price = df['high'].iloc[entry_idx]
    #     exit_price = df['low'].iloc[-1]
    #     entry_date = df.index[entry_idx]
    #     exit_date = df.index[-1]
    #     ret = (exit_price - entry_price) / entry_price
    #     hold_time = (exit_date - entry_date).days
    #     trades.append({
    #         'entry_date': entry_date,
    #         'exit_date': exit_date,
    #         'return': ret,
    #         'hold_time_days': hold_time
    #     })

    # Build daily equity curve
    positions = [0 for _ in range(len(df))]
    for t in trades:
        for i in range(t['entry_idx'], t['exit_idx']):
            positions[i] = 1

    df['positions'] = positions
    equity = [1.0]
    for i in range(1, len(df)):
        if df['positions'].iloc[i - 1] == 1:
            daily_ret = (df['close'].iloc[i] - df['close'].iloc[i - 1]) / df['close'].iloc[i - 1]
            equity.append(equity[-1] * (1 + daily_ret))
        else:
            equity.append(equity[-1])
    equity = pd.Series(equity, index=df.index)

    # Max drawdown
    # rolling_max = equity.cummax()
    # drawdown = (equity - rolling_max) / rolling_max
    # max_drawdown = drawdown.min()
    max_drawdown = 0
    for t in trades:
        max_drawdown = min(max_drawdown, t['return']) if min(max_drawdown, t['return']) < max_drawdown else max_drawdown

    total_trades = len(trades)
    avg_return = np.mean([t['return'] for t in trades]) if trades else 0
    avg_hold = np.mean([t['hold_time_days'] for t in trades]) if trades else 0

    years = (df.index[-1] - df.index[0]).days / 365.25
    trades_per_year = total_trades / years if years > 0 else 0

    return {
        'total_trades': total_trades,
        'average_return': avg_return,
        'average_hold_days': avg_hold,
        'max_drawdown': max_drawdown,
        'trades_per_year': trades_per_year,
        'equity_curve': equity,
        'trades': trades,
        'positions': positions
    }

In [ ]:
def aggregate_multi_crypto_stats(data_dict, strategy_func, exp_t=20, ema_t=100, slope_t=14, bb_t=20, bb_width=1.5, mom_thr=0.02, sell_ema=25):
    """
    data_dict: dict of ticker -> DataFrame
    strategy_func: one of strategy1, strategy2, strategy3 functions that add 'positions' column
    
    Returns aggregated stats across all cryptos.
    """
    from collections import defaultdict
    import numpy as np
    import pandas as pd

    trade_counts_per_day = defaultdict(int)
    all_trades = []
    back_tickers = []
    
    total_equity = None
    
    for ticker, df in data_dict.items():
        strat_df = strategy_func(df, exp_t, ema_t, slope_t, bb_t, bb_width, mom_thr, sell_ema)
        stats = backtest_signals(strat_df)
        all_trades.extend(stats['trades'])
        for _ in range(len(stats['trades'])):
            back_tickers.append(ticker)
        
        # Count active trades per day
        # for i, pos in enumerate(stats['positions']):
        #     if pos == 1:
        #         trade_counts_per_day[stats.index[i]] += 1
        
        # Sum equity curves by aligning on dates
        if total_equity is None:
            total_equity = stats['equity_curve']
        else:
            total_equity = total_equity.add(stats['equity_curve'], fill_value=0)
    
    # Average number of active trades per day
    daily_active_counts = pd.Series(trade_counts_per_day)
    avg_active = daily_active_counts.mean()
    max_active = daily_active_counts.max()
    
    total_trades = len(all_trades)
    avg_return = np.mean([t['return'] for t in all_trades]) if all_trades else 0
    avg_hold = np.mean([t['hold_time_days'] for t in all_trades]) if all_trades else 0
    
    # Calculate max drawdown on combined equity (average of cryptos)
    if total_equity is not None:
        total_equity /= len(data_dict)  # average equity across cryptos
        rolling_max = total_equity.cummax()
        drawdown = (total_equity - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
    else:
        max_drawdown = 0
    
    # Years based on combined date range
    all_dates = pd.concat([df.index.to_series() for df in data_dict.values()])
    total_days = (all_dates.max() - all_dates.min()).days
    years = total_days / 365.25 if total_days > 0 else 1
    trades_per_year = total_trades / years if years > 0 else 0

    # === New Metrics ===
    # 3. Sharpe Ratio
    daily_returns = total_equity.pct_change().dropna()
    sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(365) if len(daily_returns) > 1 else 0

    # 4. Sortino Ratio
    downside_std = daily_returns[daily_returns < 0].std()
    sortino_ratio = (daily_returns.mean() / downside_std) * np.sqrt(365) if downside_std > 0 else 0

    # 5. Win Rate
    win_rate = np.mean([1 if t['return'] > 0 else 0 for t in all_trades]) if all_trades else 0
    # 6. Profit Factor
    profits = [t['return'] for t in all_trades if t['return'] > 0]
    losses = [abs(t['return']) for t in all_trades if t['return'] < 0]
    profit_factor = sum(profits) / sum(losses) if losses else float('inf')

    return {
        'total_trades': total_trades,
        'average_return': avg_return,
        'average_hold_days': avg_hold,
        'max_drawdown': max_drawdown,
        'trades_per_year': trades_per_year,
        'average_active_trades_per_day': avg_active,
        'max_active_trades': max_active,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'win_rate': win_rate,
        'profit_factor': profit_factor
    }, all_trades, back_tickers
    print(all_trades)
    print(len(all_trades))
    # max_loss = min([t['return'] for t in all_trades]) if len(all_trades) > 0 else -1
    # if len(all_trades) < 50:
    #     return 0, 0, -1
    # return win_rate, avg_hold, max_loss


In [ ]:
agg_stats, trades, tick = aggregate_multi_crypto_stats(data, strategy1_exp_momentum_trend_volatility, exp_t=20, ema_t=125, slope_t=20, bb_t=25, bb_width=1.8, mom_thr=0.075, sell_ema=50)
# print(aggregate_multi_crypto_stats(data, strategy1_exp_momentum_trend_volatility, exp_t=20, ema_t=125, slope_t=20, bb_t=25, bb_width=1.5, mom_thr=0.075))
trades = pd.DataFrame(trades)
trades["tckr"] = tick
print("Metrics: ")
print("##########")
for x in agg_stats.items():
    print(x[0], ": ", x[1])

In [ ]:
trades.sort_values(by=["return"], ascending=True)

In [ ]:
strat_df = strategy1_exp_momentum_trend_volatility(data['BTCUSDT'])
stats = backtest_signals(strat_df)

In [ ]:
strat_df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import product
fig1 = go.Figure()
# Plot 1: Close (primary) and Volume (secondary)
fig1.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["close"], name="Close", line=dict(color='blue'))
)
fig1.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["volume"], name="Volume", marker=dict(color='red'), opacity=0.5)
)
fig1.add_trace(
    go.Scatter(
        x=strat_df.index[strat_df["signal"] == 1], y=strat_df.loc[strat_df["signal"] == 1, "close"],
        mode='markers', name='Signal',
        marker=dict(color='red', size=6, symbol='circle')
    )
)
fig1.update_layout(
    title="Close and Volume",
    yaxis=dict(title="Close"),
    yaxis2=dict(title="Volume", overlaying='y', side='right')
)
fig1['data'][1]['yaxis'] = 'y2'
fig1.show()

# Plot 2: Close and BB Higher (same axis)
fig2 = go.Figure()
fig2.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["close"], name="Close", line=dict(color='blue'))
)
fig2.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["BB_upper"], name="BB Higher", line=dict(color='green', dash='dash'))
)
fig2.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["BB_lower"], name="BB Lower", line=dict(color='red', dash='dash'))
)
fig2.add_trace(
    go.Scatter(
        x=strat_df.index[strat_df["signal"] == 1], y=strat_df.loc[strat_df["signal"] == 1, "close"],
        mode='markers', name='Signal',
        marker=dict(color='red', size=6, symbol='circle')
    )
)
fig2.show()



# Plot 3: Close and BB Higher (same axis)
fig3 = go.Figure()
fig3.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["close"], name="Close", line=dict(color='blue'))
)
fig3.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["EMA100"], name="EMA100", line=dict(color='green', dash='dash'))
)
fig3.add_trace(
    go.Scatter(
        x=strat_df.index[strat_df["signal"] == 1], y=strat_df.loc[strat_df["signal"] == 1, "close"],
        mode='markers', name='Signal',
        marker=dict(color='red', size=6, symbol='circle')
    )
)
fig3.show()

# Plot 4: Close (primary) and Volume (secondary)
fig4 = go.Figure()
fig4.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["close"], name="Close", line=dict(color='blue'))
)
fig4.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["exp_momentum"], name="Momentum", marker=dict(color='red'), opacity=0.5)
)
fig4.add_trace(
    go.Scatter(
        x=strat_df.index[strat_df["signal"] == 1], y=strat_df.loc[strat_df["signal"] == 1, "close"],
        mode='markers', name='Signal',
        marker=dict(color='red', size=6, symbol='circle')
    )
)
fig4.update_layout(
    title="Close and Momentum",
    yaxis=dict(title="Close"),
    yaxis2=dict(title="Momentum", overlaying='y', side='right')
)
fig4['data'][1]['yaxis'] = 'y2'
fig4.show()

# Plot 5: Close (primary) and Volume (secondary)
fig5 = go.Figure()
fig5.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["close"], name="Close", line=dict(color='blue'))
)
fig5.add_trace(
    go.Scatter(x=strat_df.index, y=strat_df["slope"], name="Slope", marker=dict(color='red'), opacity=0.5)
)
fig5.add_trace(
    go.Scatter(
        x=strat_df.index[strat_df["signal"] == 1], y=strat_df.loc[strat_df["signal"] == 1, "close"],
        mode='markers', name='Signal',
        marker=dict(color='red', size=6, symbol='circle')
    )
)
fig5.update_layout(
    title="Close and Slope",
    yaxis=dict(title="Close"),
    yaxis2=dict(title="Slope", overlaying='y', side='right')
)
fig5['data'][1]['yaxis'] = 'y2'
fig5.show()


In [ ]:
import optuna

# Optuna objective wrapper
def objective(trial):
    exp_t = trial.suggest_int('exp_t', 5, 50) 
    ema_t = trial.suggest_int('ema_t', 50, 250) 
    slope_t = trial.suggest_int('slope_t', 3, 50) 
    bb_t = trial.suggest_int('bb_t', 3, 50) 
    bb_width = trial.suggest_float('bb_width', 0.0, 5.0)
    mom_thr = trial.suggest_float('mom_thr', 0.0, 0.5)
    sell_ema = trial.suggest_int('sell_ema', 7, 120)
    return aggregate_multi_crypto_stats(data, strategy1_exp_momentum_trend_volatility, exp_t, ema_t, slope_t, bb_t, bb_width, mom_thr, sell_ema)
    # return aggregate_multi_crypto_stats(data, strategy1_exp_momentum_trend_volatility, 20, 125, 20, 25, bb_width, 0.075, sell_ema)

# Run optimization
study = optuna.create_study(directions=['maximize', 'maximize', 'maximize'])
study.optimize(objective, n_trials=50)

In [ ]:
print("Best parameters:", study.best_trials)

In [ ]:
def backtest_with_max_active(data_dict, strategy_func, max_active=5):
    """
    Executes trades 1 day after signals:
    - Buys at next day's high
    - Sells at next day's low
    """
    all_trades = []

    # First, apply strategy and calculate strength
    processed = {}
    for ticker, df in data_dict.items():
        strat_df = strategy_func(df).copy()
        if 'RSI' in strat_df.columns:
            strat_df['strength'] = strat_df['RSI']
        elif 'volume' in strat_df.columns and 'roll20_vol_avg' in strat_df.columns:
            strat_df['strength'] = strat_df['volume'] / strat_df['roll20_vol_avg']
        elif 'BB_upper' in strat_df.columns:
            strat_df['strength'] = strat_df['close'] - strat_df['BB_upper']
        else:
            strat_df['strength'] = 0
        processed[ticker] = strat_df

    # Collect all signals
    signals = []
    for ticker, df in processed.items():
        for date, row in df.iterrows():
            signals.append({
                'date': date,
                'ticker': ticker,
                'signal': row['signal'],
                'strength': row['strength'],
                'close': row['close'],
                'high': row.get('high', row['close']),
                'low': row.get('low', row['close']),
            })
    signals_df = pd.DataFrame(signals).sort_values('date')
    dates = sorted(set(signals_df['date']))

    active_trades = {}
    equity = []
    capital = 1.0
    capital_per_trade = capital / max_active

    for i in range(len(dates) - 1):  # avoid out-of-bounds for i+1
        current_date = dates[i]
        next_date = dates[i + 1]
        today_signals = signals_df[signals_df['date'] == current_date]

        # Process exits
        exit_tickers = today_signals[
            (today_signals['signal'] == -1) & 
            (today_signals['ticker'].isin(active_trades.keys()))
        ]['ticker'].tolist()

        for t in exit_tickers:
            df = processed[t]
            if next_date in df.index:
                exit_price = df.loc[next_date, 'low']
                entry_price = active_trades[t]['entry_price']
                ret = (exit_price - entry_price) / entry_price
                capital += capital_per_trade * ret
                all_trades.append({
                    'ticker': t,
                    'entry_date': active_trades[t]['entry_date'],
                    'exit_date': next_date,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'return': ret
                })
                del active_trades[t]

        # Process entries
        free_slots = max_active - len(active_trades)
        buy_signals = today_signals[
            (today_signals['signal'] == 1) & 
            (~today_signals['ticker'].isin(active_trades.keys()))
        ]

        if free_slots > 0 and len(buy_signals) > 0:
            buy_signals = buy_signals.sort_values('strength', ascending=False).head(free_slots)
            for _, row in buy_signals.iterrows():
                t = row['ticker']
                df = processed[t]
                if next_date in df.index:
                    entry_price = df.loc[next_date, 'high']
                    active_trades[t] = {
                        'entry_date': next_date,
                        'entry_price': entry_price
                    }

        # Compute daily return from open trades (approx, based on close-close)
        daily_return = 0
        for t, trade in active_trades.items():
            df = processed[t]
            if current_date in df.index and dates[i - 1] in df.index:
                today_close = df.loc[current_date, 'close']
                prev_close = df.loc[dates[i - 1], 'close']
                ret = (today_close - prev_close) / prev_close
                daily_return += capital_per_trade * ret

        capital += daily_return
        equity.append({'date': current_date, 'capital': capital, 'open_trades': len(active_trades)})

    equity_df = pd.DataFrame(equity).set_index('date')
    cum_return = capital - 1
    equity_series = equity_df['capital']
    rolling_max = equity_series.cummax()
    drawdown = (equity_series - rolling_max) / rolling_max
    max_dd = drawdown.min()

    return {
        'equity_curve': equity_df,
        'cumulative_return': cum_return,
        'max_drawdown': max_dd,
        'max_active_trades': max(equity_df['open_trades']),
        'trades': all_trades
    }


In [ ]:
import matplotlib.pyplot as plt
# --- Run backtest ---
results = backtest_with_max_active(data, strategy1_exp_momentum_trend_volatility, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')
plt.title("Relative Performance: Strategy vs Bitcoin vs SPY")
plt.ylabel("Normalized Return (start = 1.0)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
pd.DataFrame(results["trades"])